# Tarea 2 - Funciones hash y cifrado por bloques

1. Se recomienda hacer en equipo (de hasta 4 personas) para poder discutir los problemas y repartir el trabajo.
2. Para tu entrega puedes realizar los programas en esta libreta de Jupyter o puedes anexar tus programas en archivos separados. En cualquier caso, agrega comentarios que expliquen el funcionamiento.
3. Cualquier tarea plagiada total o parcialmente se calificará con cero. 

**Alumnos:**

-García Gómez Jazmín

-Barredo Escalona Paola Betsabe

-Luna Piña Erick Eduardo

-Altamirano Niño Luis Enrique

## 1. Vacuna para ransomware (1.5 puntos)

Extrae el archivo `mis_archivos.zip`, que contiene el directorio `Mis archivos`. Desde este directorio ejecuta `juego.py` con Python 3:
```		
Mis archivos$ python3 juego.py
```
Este programa es un ransomware de juguete, encripta todos los archivos del directorio actual y pide una recompensa para poder recuperar los originales. Haz un programa que funcione como vacuna para el ransomware, es decir, que revierta los cambios que fueron hechos al ejecutar `juego.py`. Cuando se ejecute tu programa en un directorio donde anteriormente se ejecutó el ransomware, se recuperarán los archivos originales (tal como eran, sin ninguna diferencia). Explica y justifica el funcionamiento de tu programa vacuna.

**Respuesta:**

La clave para desarrollar la vacuna fue que conocíamos que la cadena de bytes $d_b$  que se escribía en el archivo oculto *'.xyz'* era tal que $\text{int.from_bytes}(d_b) = \text{key}\lor 1$ donde $\text{key}$ es la llave que se usó para cifrar los archivos. Después usando el razonamiento de que un $\text{or}$ de un número cualquiera con 1 solo afecta al bit más a la derecha, se tiene que $\text{int.from_bytes}(d_b) = \text{key}- 1$ o $\text{int.from_bytes}(d_b) = \text{key}$, de ahí solo tuvimos que leer el arhcivo oculto para obtener el valor de $d_b$ y después la llave($\text{key}$).

A continuación se muestra el código de la vacuna, debe ejecutarse en el mismo directorio que donde se ejecutó el ransomware.

In [ ]:
#vacuna para el ransomware
# -*- coding: utf-8 -*-
from sys import argv,version_info
import os
assert version_info[0] == 3, 'USA PYTHON 3'

print('Descifrando archivos...')
#Leemos el archivo oculto generado por el ransomware
with open('.xyz','rb') as z:
    c = int.from_bytes(z.read(1),'little')
    d = int.from_bytes(z.read(32),'big')
    k = int.from_bytes(z.read(32),'big')

#verificamos si la llave es de la forma k=d-1 o k=d
if(k == d*(d-1)%(1<<c)):
  key = d -1
else:
  key = d

#Pasamos la llave a bytes
keyb = key.to_bytes(16,'big')

#Definimos la funcion para descifrar un archivo
def descifra(x):
    with open(x.replace('.enc',''),'wb') as z:
        #Hacemos xor entre la llave y los bytes del archivo
        z.write((lambda x:bytes([x[i]^keyb[i%16] for i in range(len(x))])) (open(x,'rb').read()))
    #eliminamos el archivo en su version cifrada    
    os.remove(x)

#obtenemos los nombres de los archivos en el directorio
_,_,x = next(os.walk('./'))
#excluimos el archivo de la vacuna
x.remove(argv[0])
#removemos el archivo del ransomware
x.remove('juego.py')
#Desciframos todos los archivos del directorio
list(map(descifra,x))
print('\n Archivos descifrados <3')

149090576740855166249891003669300651085
La llave:  149090576740855166249891003669300651084
.x
216
18263894827650607574052860164329200729767996094524250675998989020
k-bytes b'\x00\x00\x00\x00\x00,e\xa6O\xc3\x13\xed~\xf8o\x1e\xd0\x17<\x8b\xb3-\x94\xfc\tSC\x85\xf8e\x9a\xdc'
b'\xd8'
k_arch: 18263894827650607574052860164329200729767996094524250675998989020
d_arch: 149090576740855166249891003669300651085
c_arch: 216
False
True
key 149090576740855166249891003669300651084
112
llave:  149090576740855166249891003669300651084


## Funciones hash

In [ ]:
from hashlib import sha1

datos = b'datos datos xD'

print('Digesto como bytes')
digesto = sha1(datos).digest()
print(digesto)

print('\nDigesto en hexadecimal')
digesto_hex = sha1(datos).hexdigest()
print(digesto_hex)

Digesto como bytes
b'\xd1\x19\xec\x92\x00\x1ba[\xd5\xe8\xd9\xfc\xce\xe4\xcbhU\x86\x9d\t'

Digesto en hexadecimal
d119ec92001b615bd5e8d9fccee4cb6855869d09


## 2. Contraseñas en bases de datos (1.5 puntos)

1. El archivo `BD_jaqueada.txt` contiene una lista con los datos de 500 usuarios de un servicio web. La lista incluye el nombre de usuario y el hash de su contraseña. Por la forma de las cadenas en hexadecimal haces la suposición de que cada cadena es de la forma `$salt$H(contraseña||salt)` o `$salt$H(salt||contraseña)`, donde H es una función hash popular con salida de 32 bytes. Usando la lista de contraseñas conocidas `passwords.gz` que se adjunta con este documento, haz un programa para encontrar las contraseñas de los usuarios (por lo menos 50 contraseñas por cada miembro del equipo). Toma en cuenta que la búsqueda puede llevar un buen rato, así que reduce lo más que puedas la cantidad de operaciones realizadas durante la búsqueda. **Advertencia:** el archivo de contraseñas conocidas es muy grande, ten cuidado al usarlo ya que tus programas o incluso tu computadora podrían morir si se acaba la memoria disponible.


**Respuesta:**

Lo primero que se hizo fue checar con el primer hash del archivo `BD_jaqueada.txt` y cada una de las contraseñas del archivo `passwords.gz`, que funcion hash se estaba usando para poder definir después nuestra propia función para descifrar. Para el `usuario0001` la contraseña es `torie2` y sabemos que se usó `sha256`. Sabiendo eso, procedimos a programar una funcion para tratar de descifrar las contraseñas de los usuarios que aparecen en `BD_jaqueada.txt`. El siguiente es el código que se usó para obtener esto:

In [ ]:
from hashlib import sha256

#abrimos los archivos de contraseñas conocidas y de la base de datos con los hash.
with open('realhuman_phill.txt','rb') as passwords, open('BD_jaqueada.txt','rt') as bd:
 
    #Obtiene una contraseña del archivo de contraseñas conocidas
    def getPassword():
        byte =  b''
        h = passwords.read(1)
        #cuando encontramos un salto de linea terminamos de construir la contraseña
        while h!=b'\n':
            h = passwords.read(1)
            #quitamos los saltos de linea de las contraseñas
            if h!=b'\n':
                byte += h
        return byte
    
    #obtiene un hash del archivo de la base de datos y nos regresa los datos importantes
    def getHash():
        line = bd.readline().split(' ')
        user = line[0]
        hash = line[1].replace('\n','')
        splhash = hash.split('$')
        salt = splhash[1]
        digest = splhash[2]
        return user,salt,digest
    
    #descifra las contraseñas siempre y cuando aparezcan en el archivo de contraseñas conocidas
    def decryptPassword():
        #Creamos un archivo para guardar las contraseñas encontradas
        with open('decryptedpasswords.txt','wt') as dc:
            #recorremos los usuarios(500 en total)
            for _ in range(500):
                user,salt,digest = getHash()
                #mostramos en consola que usuario estamos revisando
                print(user)
                currentPassword = b'\n'
                #recorremos las contraseñas conocidas(en total son 63941069)
                for _ in range(63941069):
                    #verificamos si la contraseña actual concatenada con el salt nos da el digest necesario
                    #en tal caso guardamos esa contraseña en el archivo
                    if sha256(bytes.fromhex(salt)+currentPassword).hexdigest() == digest:
                        print('sha256 salt||password:', currentPassword)                       
                        dc.write(user + ' ' + currentPassword.decode() + '\n')
                        break
                    if sha256(currentPassword+bytes.fromhex(salt)).hexdigest() == digest:
                        print('sha256 password||salt:', currentPassword)
                        dc.write(user + ' ' + currentPassword.decode() + '\n')
                        break
                    currentPassword = getPassword()
                #regresamos al inicio del archivo de contraseñas conocidas    
                passwords.seek(0)
    
    #intentamos descifrar las contraseñas
    decryptPassword()

Cada miembro del equipo se encargó de buscar para 100 usuarios, muchos usuarios no tenían una contraseña conocida,los datos que pudimos obtener fueron:

usuario       contraseña

`usuario0001` `b'torie2'`

`usuario0012` `b'anky'`

`usuario0058` `b'BASINGER'`

`usuario0092` `b'danalex'`

`usuario0095` `b'ptrol'`

`usuario0097` `b'e+b'`

`usuario0118` `b'571978'`

`usuario0134` `b'ESTIMONY'`

`usuario0161` `b'GLIMMER'`

`usuario0186` `b'reitwiesn'`

`usuario0232` `b'tramore'`

`usuario0252` `b'annaho'`

`usuario0282` `b''`

`usuario0303` `b'algi'`

`usuario0349` `b'issa1'`

`usuario0367` `b'ARICELLA'`

`usuario0369` `b'00286'`

`usuario0371` `b'iddare'`

`usuario0405` `b'cfuxqtald'`

`usuario0415` `b'evil-dispelling'`

`usuario0422` `b'ATSON1'`

`usuario0436` `b'ANALYTIQUE'`

`usuario0439` `b'mobolade'`









2. En otra base de datos se usó el algoritmo `scrypt` para obtener el hash de las contraseñas, pues con este algoritmo se dificulta la búsqueda mediante listas de contraseñas comunes. `scrypt` tiene varios parámetros que se pueden cambiar para incrementar el costo (en procesador y memoria) de calcular un hash. La biblioteca hashlib incluye `scrypt`, pruébalo y después, usando los parámetros $n=2^{16}, r=8, p=1$ y la cadena salt `0xd8201aae236713fefe9a5266dc1f8012`, encuentra la contraseña de juanito. Es posible que tengas que pasar el parámetro `maxmem=2**30`.
```
juanito 5f495364792782144918397bdbb72bc04326a883138a11f3d0b61a3d2576ca00
```

In [ ]:
#Probando el algoritmo scrypt
from hashlib import scrypt

scrypt(b'hola',salt=bytes.fromhex(hex(0xd8201aae236713fefe9a5266dc1f8012)[2:]),n=2**16,r=8,p=1,maxmem=2**30)

b"x\x12\x0b\xa8\x0c\xe5\xd6\xef\x1b\xbaJE\x10I\x9c\xe0[\xe1k\xf5\x93\xd2\xbbT\xcb\x0e\x10V\xc4a\x06\x12\x9a\x8bo\xe4\xee\x14\x95$W\x92\xc8\x9d\x192e`:\xddS\x9a\xf9\x86{\xe4\xb0\xcb'r\xe3\x0cq,"

In [ ]:
#codigo para descifrar la contraseña de juanito
from hashlib import scrypt
with open('realhuman_phill.txt','rb') as passwords:

    def getPassword():
        byte =  b''
        h = passwords.read(1)   
        while h!=b'\n':
            if h!=b'\n':
                byte += h
            h = passwords.read(1)
        return byte
    
    currentPassword = b'\n'
    saltj = 0xd8201aae236713fefe9a5266dc1f8012
    saltj = bytes.fromhex(hex(saltj)[2:])
    hashj = '5f495364792782144918397bdbb72bc04326a883138a11f3d0b61a3d2576ca00'

    for i in range(63941070):
        if scrypt(b'hola',salt=saltj,n=2**16,r=8,p=1,maxmem=2**30)==hashj:
            print('contraseña de juanito:', currentPassword)
            break
        currentPassword = getPassword()

## 3. Minando pumacoins (1.5 puntos)

El pumacoin (PMC) es una criptomoneda (100% real no fake) similar al Bitcoin, pero que en vez de SHA256 utiliza una función hash más moderna llamada BLAKE2. BLAKE2 es una actualización de BLAKE, función hash que participó en la competencia para definir el estándar SHA3 pero que perdió frente a las funciones esponja (Keccak). A pesar de no ser un estándar, la seguridad y velocidad de BLAKE2 han permitido que sea adoptada en bastantes aplicaciones. La biblioteca hashlib de las últimas versiones de Python incluye una implementación de BLAKE2.

Para generar un pumacoin, la especificación del protocolo dice que se usa una *prueba de trabajo* definida de la siguiente manera:

Dado el identificador de un bloque de transacciones `id`, de longitud 16 bytes, los mineros tienen el reto de encontrar una cadena `x` de 16 bytes tal que el digesto generado por `BLAKE2s-256(id||x)` tiene al principio una cadena de bytes especial. Dicha cadena depende de la cantidad de mineros participando en la red: si hay a lo más 100 mineros la cadena especial es `0x242424`, cuando 100 < núm. de mineros < 5000 la cadena  es `0xf09fa491` y cuando hay 5000 o más mineros la cadena especial es `0xe29a92e29898`. El primer minero que encuentra `x` verifica que el bloque es correcto y comunica a los demás mineros que encontró `x`, a este minero se le recompensa con 1 PMC.

1. Implementa una función para minar pumacoins. Recibe como entrada el `id` del bloque y `N` el número de mineros en un momento dado, y como salida da la cadena `x`.
2. Resuelve la prueba de trabajo para las entradas `(0xd1c5593465eb5bfb9fcad9adf90af61f, 50)` y `(0x73bf71c8cd6f03c414cd2477a17570c4, 1000)`. Muestra las cadenas encontradas junto con el digesto que generan.
3. (Opcional) Intenta con la entrada `(0x68188585019b02d746b48b4d06c15dcf, 5000)`, esto llevará más tiempo dependiendo de la capacidad de tu computadora (y de tu suerte).

In [ ]:
#3.1: IMPLEMENTACIÓN DE LA FUNCIÓN PARA MINAR PUMACOINS
from hashlib import blake2s
import os

def minapumacoins(id,N):
  #revisamos cual es la cadena especial a encontrar
  if(N<=100):
    especial = hex(0x242424)[2:]
  elif(N in range(101,5000)):
    especial = hex(0xf09fa491)[2:]
  else:
    especial = hex(0x29ea92e29898)[2:]

  #generamos una cadena aleatoria de 16 bytes 
  x = os.urandom(16)
  i = False
  digesto = ''
  #buscamos un valor de x tal que blake2s(x||id) contenga la cadena especial al inicio
  while i!=True:
    x = os.urandom(16)
    digesto = blake2s(bytes.fromhex(hex(id)[2:])+x).hexdigest()
    if digesto.find(especial,0,12)==0:
      i = True

  print("X=",x)
  print("Digesto=",digesto)
  return x

In [ ]:
#3.2.1
minapumacoins(0xd1c5593465eb5bfb9fcad9adf90af61f, 50)

X= b'\x82\x1a\x86U\xeb\x8e\xcd;\xabg\x1d\xd06U2\xb3'
Digesto= 24242440f962e4862e57add814c95509a87dcf8096492f0e9963278265a7ed31


b'\x82\x1a\x86U\xeb\x8e\xcd;\xabg\x1d\xd06U2\xb3'

**Respuesta:**

$x=$ `b'\x82\x1a\x86U\xeb\x8e\xcd;\xabg\x1d\xd06U2\xb3'`

In [ ]:
#3.2.2
minapumacoins(0x73bf71c8cd6f03c414cd2477a17570c4, 1000)

**Respuesta:**

$x=$ `b'\x0bw\xba\xea^\x01\x03\xa8YY\x1b^\x85\xfe\xc8\xb8'`

In [ ]:
#3.3
minapumacoins(0x68188585019b02d746b48b4d06c15dcf, 5000)

## Modos de operación en cifrado por bloques

Recordemos que los cifradores por bloques solamente funcionan con mensajes de una longitud fija (longitud de bloque). Para poder encriptar un mensaje de longitud arbitraria lo que se hace es
1. Agregar un relleno al final del mensaje para que su longitud sea un múltiplo de la longitud de bloque.
2. Dividir el mensaje en bloques.
3. Aplicar el algoritmo de cifrado.

El paso 3 puede hacerse de distintas maneras, a cada de una de ellas le llamamos **modo de operación**. Existen muchos modos de operación, varios de ellos están definidos en estándares, pero en esta tarea solamente abordaremos tres de ellos.

#### Modo ECB

El modo de operación más directo e *inseguro* se conoce como Electronic Codebook o ECB. Consiste en inicializar el algoritmo con la llave *k* y aplicarlo en cada bloque del mensaje. Más detalladamente, si $E_k$ es nuestro algoritmo cifrador inicializado con la llave $k$ y tenemos el mensaje $m = m_1m_2\cdots m_n$, donde cada $m_i$ es un bloque, obtenemos el mensaje cifrado $c = c_1c_2\cdots c_n$ donde $c_i = E_k(m_i)$.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d6/ECB_encryption.svg/600px-ECB_encryption.svg.png "Modo ECB")

En clase vimos que un algoritmo cifrador determinista no es seguro, ya que al aplicarlo a mensajes claros iguales se obtienen mensajes cifrados iguales. De la misma manera, este modo de operación mantiene los patrones que se pueden encontrar en los bloques del mensaje claro. Para arreglar este problema, los modos de operación posteriores utilizan una cadena extra que permite *aleatorizar* el mensaje.

In [ ]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB

key = os.urandom(16)
#key = b'Mi llave secreta'
# Se inicializa AES con la llave y el modo ECB
aes_k = Cipher(AES(key), ECB())
enc = aes_k.encryptor()
# Mensaje de tres bloques de longitud
mensaje = b'un bloke secreto' + b'-'*16 + b'un bloke secreto'
mensaje_cifrado = enc.update(mensaje) + enc.finalize()
print('Bloques cifrados')
print(mensaje_cifrado[:16].hex(), mensaje_cifrado[16:32].hex(), mensaje_cifrado[32:].hex())

dec = aes_k.decryptor()
mensaje_descifrado = dec.update(mensaje_cifrado) + dec.finalize()
print('Mensaje descifrado')
print(mensaje_descifrado)

#### Modo CBC
El modo de operación Cipher block chaining o CBC usa un *vector de inicialización* $IV$ para aleatorizar el mensaje claro. Para encriptar $m = m_1m_2\cdots m_n$, definimos $c_0 = IV$ y obtenemos el mensaje cifrado $c = c_0c_1c_2\cdots c_n$, donde $c_i = E_k(m_i\oplus c_{i-1})$; nota que $IV$ se agrega al inicio del mensaje cifrado para que pueda ser usado a la hora de descifrar.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/CBC_encryption.svg/640px-CBC_encryption.svg.png "Modo CBC")

In [ ]:
from cryptography.hazmat.primitives.ciphers.modes import CBC

key = os.urandom(16)
iv = os.urandom(16)
# Se inicializa AES con la llave y el modo CBC, que requiere un IV
aes_k = Cipher(AES(key), CBC(iv))
enc = aes_k.encryptor()
mensaje = b'un bloke secreto' + b'-'*16 + b'un bloke secreto'
mensaje_cifrado = enc.update(mensaje) + enc.finalize()
print('Bloques cifrados')
print(mensaje_cifrado[:16].hex(), mensaje_cifrado[16:32].hex(), mensaje_cifrado[32:].hex())

dec = aes_k.decryptor()
mensaje_descifrado = dec.update(mensaje_cifrado) + dec.finalize()
print('Mensaje descifrado')
print(mensaje_descifrado)

#### Modo CTR

En el modo CTR (de contador) el algoritmo de cifrado se aplica a una cadena $IV||ctr$, donde $IV$ es un vector de inicialización generado aleatoriamente y $ctr$ es una cadena que sirve como contador (el símbolo $||$ se refiere a concatenación). La cadena $IV||ctr$ será la entrada de $E_k$ y el valor del contador se incrementa en uno por cada bloque del mensaje claro. El mensaje cifrado es el XOR de los bloques claros y la salida correspondiente de $E_k$, es decir, el mensaje claro $m = m_1m_2\cdots m_n$ se cifra en el mensaje $c = c_0c_1\cdots c_n$, donde $c_0 = IV$ y para los demás bloques $ c_i = E_k(IV||i) \oplus m_i$.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/CTR_encryption_2.svg/640px-CTR_encryption_2.svg.png "Modo CTR")

### Padding
El relleno o padding que se agrega a los bloques para que tengan el tamaño adecuado puede ser arbitrario (en tanto pueda ser quitado sin ambigüedad). En esta tarea usarás el padding que consiste en agregar la secuencia de bytes BB...B de longitud igual a B, donde B es un entero entre 1 y 16. Por ejemplo, si queremos rellenar la cadena `hola mundo` para tener un bloque de 16 bytes, se agregará la cadena de 6 bytes `0x060606060606`, o la cadena `A` se rellena con la cadena `0x0f0f0f0f0f0f0f0f0f0f0f0f0f0f0f`.

## 4. Implementando modos de operación (1.5 puntos)

Investiga el proceso de descifrado para los modos CBC y CTR (puedes tratar de deducirlo revisando los diagramas mostrados arriba). Luego implementa las funciones de cifrado y descifrado de los modos CBC y CTR usando AES con llaves de 128 bits, para eso vas a utilizar el modo ECB como base para poder aplicar $AES_k$ o $AES_k^{-1}$ según corresponda. Tus funciones aceptarán mensajes de tamaño arbitrario, por lo cual tienen que agregar un relleno según sea necesario. Las funciones de descifrado devolverán el texto claro solamente si el padding del mensaje es correcto, en caso contrario lanzarán una excepción.  El objetivo es que tú programes los modos de operación, por lo cual no puedes usar los modos CBC y CTR que vienen incluidos en bibliotecas.

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB

def aes128_cbc_enc(llave, mensaje):
    aes = Cipher(AES(key), ECB())
    enc = aes.encryptor()
    # FALTA IMPLEMENTAR

def aes128_cbc_dec(llave, cifrado):
    # FALTA IMPLEMENTAR
    
def aes128_ctr_enc(llave, mensaje):
    # FALTA IMPLEMENTAR
    
def aes128_ctr_dec(llave, cifrado):
    # FALTA IMPLEMENTAR

In [ ]:
# Para hacer pruebas
llave = b'Mi llave secreta'
mensaje = b'Mensaje de texto mas grande que un bloque y de longitud que si es multiplo de 16'
mensaje2 = b'UwU'